In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading PL data
PL_dict = zload(data_folder + 'PL_dict.pkz') 

In [5]:
# PL_dict

In [6]:
mu_1 = PL_dict['PL_AM']  # normal PL for AM period
mu_2 = PL_dict['PL_MD']  # normal PL for MD period
mu_3 = PL_dict['PL_PM']  # normal PL for PM period
mu_4 = PL_dict['PL_NT']  # normal PL for NT period

mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

mu_2 = mu_adjust(mu_2)  # normal PL
mu_02, mu2, mu_12, P2, G_12, H_12, U_12 = ChainGen_(mu_2)

mu_3 = mu_adjust(mu_3)  # normal PL
mu_03, mu3, mu_13, P3, G_13, H_13, U_13 = ChainGen_(mu_3)

mu_4 = mu_adjust(mu_4)  # normal PL
mu_04, mu4, mu_14, P4, G_14, H_14, U_14 = ChainGen_(mu_4)

<string>:486: RuntimeWarning: covariance is not positive-semidefinite.


In [7]:
beta = 0.001
G_list = [G_11, G_12, G_13, G_14]
H_list = [H_11, H_12, H_13, H_14]
U_list = [U_11, U_12, U_13, U_14]

eta_wc = {}
eta_Sanov = {}

# Get thresholds for Hoeffding's test corresponding to sample length n  
def etaWC(n):
    return HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)

In [8]:
# Loading test data

with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [9]:
sample_path_jam_test_data_sorted_trans['0']

{u'alphaTrans': 60.0,
 u'startTime': 300.0742,
 u'uuid': [u'4a27354e-e8c6-373c-9863-f6bfe6d7a4d4',
  u'd836e23a-c958-37a7-b126-c70258b61a7a']}

In [10]:
i = 0  # index of windows
sam_gap = 120
win_size = 600
win_list = []
while i * sam_gap + win_size < 24 * 3600:
    win_list.append((i * sam_gap, i * sam_gap + win_size))
    i += 1

In [11]:
i

715

In [12]:
win_num = len(win_list)
jam_num = len(sample_path_jam_test_data_sorted_trans)

win_jam = {}
for i in range(win_num):
    key = str(i)
    value = []
    for j in range(jam_num):
        if ((sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] >= win_list[i][0]) \
            and (sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] < win_list[i][1])):
            value.append(str(j))
    win_jam[key] = value

In [13]:
win_jam['714'], win_list[714][0], win_num 

(['3733',
  '3734',
  '3735',
  '3736',
  '3737',
  '3738',
  '3739',
  '3740',
  '3741',
  '3742',
  '3743',
  '3744',
  '3745',
  '3746',
  '3747',
  '3748',
  '3749',
  '3750',
  '3751',
  '3752',
  '3753',
  '3754'],
 85680,
 715)

In [14]:
sample_path_jam_test_data_sorted_trans[win_jam['100'][0]]['startTime']

12300.2363

In [15]:
KL_list = []  # The list of the KL divergences; each is for a detection window
etaWC_list = []
for i in range(win_num):
    sample_path_jam = [sample_path_jam_test_data_sorted_trans[key]['alphaTrans'] for key in win_jam[str(i)]]
    n = len(sample_path_jam)
    if (n != 0):
        etaWC_list.append(etaWC(n))
        KL_1 = KL_est(sample_path_jam, mu_1)
        KL_2 = KL_est(sample_path_jam, mu_2)
        KL_3 = KL_est(sample_path_jam, mu_3)
        KL_4 = KL_est(sample_path_jam, mu_4)
        KL_list.append(min([KL_1, KL_2, KL_3, KL_4]))
    else:
        etaWC_list.append(1e-6)
        KL_list.append(0)

In [16]:
win_jam[str(100)], win_list[100][1]

(['225', '226', '227', '228', '229', '230'], 12600)

In [17]:
time_range = [win_list[i][0] / 60.0 for i in range(win_num)]
plot_points(time_range, KL_list, etaWC_list)
plt.ylabel('divergence')
plt.xlabel('time (min)')
pylab.ylim(-0.1, max(etaWC_list)+0.5)
pylab.xlim(0, max(time_range) + 10)
plt.savefig(data_folder + 'detec_results_WC_robust.pdf')
plt.show()

In [18]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(715, 715, 715, [0.0, 2.0])

In [19]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [20]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)

In [27]:
sample_path_jam_test_data_sorted['2917']

{u'alpha': 7.0,
 u'length': 1079.0,
 u'numPts': 32.0,
 u'speed': 3.7361111111111107,
 u'startTime': 77100.1694,
 u'uuid': u'c02948f2-e25e-3edc-85bb-85dcd34ed549'}

In [22]:
len(set(potential_ano_list)), set(potential_ano_list)

(479,
 {'1167',
  '1168',
  '1169',
  '1170',
  '1171',
  '1172',
  '1173',
  '1174',
  '1175',
  '1176',
  '1177',
  '1178',
  '1179',
  '1180',
  '1181',
  '1182',
  '1183',
  '1184',
  '1185',
  '1186',
  '1187',
  '1188',
  '1189',
  '1190',
  '1191',
  '1192',
  '1193',
  '1194',
  '1195',
  '1526',
  '1527',
  '1528',
  '1529',
  '1530',
  '1531',
  '1532',
  '1533',
  '1534',
  '1535',
  '1536',
  '1537',
  '1538',
  '1539',
  '1540',
  '1541',
  '1542',
  '1543',
  '1544',
  '1545',
  '1546',
  '1547',
  '1548',
  '1549',
  '1550',
  '1551',
  '1552',
  '1553',
  '1554',
  '1555',
  '1556',
  '1557',
  '1558',
  '1559',
  '1560',
  '1561',
  '1848',
  '1849',
  '1850',
  '1851',
  '1852',
  '1853',
  '1854',
  '1855',
  '1856',
  '1857',
  '1858',
  '1859',
  '1860',
  '1861',
  '1862',
  '1863',
  '1864',
  '1865',
  '1866',
  '1867',
  '1868',
  '1869',
  '1870',
  '1871',
  '1872',
  '1873',
  '1874',
  '1875',
  '1876',
  '1877',
  '1878',
  '1879',
  '2170',
  '2171',
  '2

In [23]:
len(sample_path_jam_test_data_sorted_trans)

3755

In [24]:
800 / 60.0, 1000 / 60.0, 1200 / 60.0, 1400 / 60.0

(13.333333333333334, 16.666666666666668, 20.0, 23.333333333333332)

In [25]:
# Refine the detection results; if a jam's length is larger than 1 km,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > 1000):
        ano_list.append(key)

In [26]:
len(ano_list), ano_list

(52,
 ['2917',
  '2314',
  '2982',
  '3179',
  '1533',
  '3206',
  '3204',
  '3209',
  '3630',
  '2949',
  '2947',
  '2946',
  '2943',
  '1195',
  '2348',
  '1178',
  '1877',
  '2887',
  '2888',
  '3620',
  '3626',
  '2979',
  '2973',
  '2985',
  '1547',
  '1544',
  '1540',
  '1188',
  '1189',
  '2190',
  '2195',
  '3164',
  '1867',
  '1866',
  '2890',
  '3183',
  '2968',
  '3613',
  '3043',
  '3042',
  '3047',
  '2325',
  '2324',
  '1554',
  '3006',
  '1855',
  '2927',
  '2926',
  '3053',
  '2180',
  '2215',
  '1181'])

In [83]:
for key in potential_ano_list:
    if (sample_path_jam_test_data_sorted[key]['startTime'] > 69900 and 
        sample_path_jam_test_data_sorted[key]['startTime'] < 69901):
        uuid = sample_path_jam_test_data_sorted[key]['uuid']
        print(points_dict_Mar16[uuid][0][2])

[-71.106477, 42.339625]
[-71.099965, 42.335753]
[-71.085783, 42.343345]
[-71.095399, 42.373042]
[-71.084797, 42.371644]
[-71.092499, 42.372662]
[-71.079863, 42.365356]
[-71.084816, 42.342892]
[-71.12384, 42.33408]
[-71.084824, 42.343007]
[-71.095326, 42.345231]
[-71.090906, 42.343896]
[-71.100016, 42.343967]
[-71.088319, 42.361188]
[-71.0752, 42.350675]
[-71.080475, 42.341624]
[-71.095718, 42.33774]
[-71.092979, 42.351272]
[-71.128311, 42.346634]
[-71.095043, 42.334793]
[-71.140266, 42.352543]
[-71.115334, 42.357787]
[-71.108124, 42.343536]
[-71.130324, 42.351925]
[-71.139905, 42.349322]
[-71.092489, 42.338849]
[-71.095492, 42.337643]
[-71.09126, 42.333495]
[-71.106398, 42.335166]
[-71.106477, 42.339625]
[-71.123558, 42.334808]
[-71.112608, 42.32708]
[-71.106729, 42.333282]
[-71.121418, 42.338137]
[-71.110713, 42.328623]
[-71.117385, 42.331674]
[-71.124749, 42.330242]
[-71.125035, 42.351532]
[-71.11797, 42.33914]
[-71.085057, 42.342663]
[-71.109346, 42.372617]
[-71.105699, 42.366589]
[

In [30]:
with open(data_folder + '/points_dict_Mar16.json', 'r') as json_file:
    points_dict_Mar16 = json.load(json_file)

In [67]:
points_dict_Mar16['50a5d058-2b16-3034-bd41-623f96b2829a']

[[[-71.123535, 42.334441],
  [-71.12364, 42.334329],
  [-71.12384, 42.33408],
  [-71.124185, 42.333598],
  [-71.124659, 42.33294],
  [-71.12478, 42.332789],
  [-71.12535, 42.33198],
  [-71.125498, 42.331798],
  [-71.125729, 42.33141],
  [-71.12573, 42.33141],
  [-71.125835, 42.331218],
  [-71.12595, 42.330951],
  [-71.126026, 42.330675],
  [-71.126057, 42.330385],
  [-71.126085, 42.330063]],
 15]

In [43]:
('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')


('-71.128121', '42.359786', 'ACCIDENT', 68700.0, '2016-03-16 19:06:00.000')

In [ ]:
jam1 = [42.359786, -71.128121]

In [90]:
jam2 = [42.363025, -71.13641] 

In [91]:
from geopy.distance import vincenty
vincenty(jam1, jam2).meters

771.8372664016924